In [3]:
# Importação das bibliotecas necessárias

import pandas as pd
import numpy as np
import glob
import os
import re
from sqlalchemy import create_engine
from datetime import datetime
import locale

In [4]:
# Estabelecimento de conexão com o banco de dados
# para extração das informações das tabelas necessárias


database_config = {
    "host": "cat42.cohe0l2miaal.us-east-1.rds.amazonaws.com",
    "user": "master",
    "password": "dms191411",
    "database": "cat"
}

# Create a connection to the MySQL database
engine = create_engine(f"mysql+pymysql://{database_config['user']}:{database_config['password']}@{database_config['host']}/{database_config['database']}")

In [5]:
# Definição da loja a ser utilizada

loja = 100

In [6]:
# Importação das planilhas necessárias para
# gerar os documentos da 000, 0150, 0200, 1050 e 1100

planilha = pd.read_excel(f'Loja_{loja}/ficha_3_loja_{loja}_v1.xlsb', sheet_name='ficha3', engine='pyxlsb')
planilha['DATA'] = pd.to_datetime(planilha['DATA'], format='%Y-%m-%d')
template = pd.read_excel('CFOP_e_template.xlsx', sheet_name='Template 150', dtype=str)
planilha

,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,ICMS_INI,...,VLR_CONFR_PCAT,SALDO_FINAL_MES_QTD,SALDO_FINAL_MES_ICMS,ALIQUOTA,VALOR,Valor Base Cálculo ICMS ST Retido Operação Anterior,Valor Complementar,Valor ICMS Substituição Tributária,Valor ICMS Operação,ICMS_TOT
0,35191071605265000757590005075110599441610811,2019-10-30,5405,1,18,1,1,1,2.0,10.192509,...,5.096254,1.0,5.11,18,20.44,0.00,NaN,0.0,0.0,0.0000
1,35191027849963000110550010005997251005978274,2019-11-01,1403,2,18,0,1,1,NaN,NaN,...,NaN,2.0,11.36,18,15.45,17.02,17.66,0.0,0.0,6.2424
2,35191271605265000757590005075110698126997891,2019-12-11,5405,1,18,1,1,1,NaN,NaN,...,5.680000,NaN,NaN,18,19.24,0.00,NaN,0.0,0.0,0.0000
3,35191271605265000757590005075110698126997891,2019-12-11,5405,4,18,1,1,1,NaN,NaN,...,5.680000,NaN,NaN,18,19.24,0.00,NaN,0.0,0.0,0.0000
4,35191227849963000110550010006416671006396546,2019-12-13,1403,2,18,0,1,2,NaN,NaN,...,NaN,2.0,12.51,18,30.90,34.14,35.33,0.0,0.0,12.5046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717557,35201127849963000110550010009983911009949320,2020-12-01,1403,1,935908,0,1,56,0.0,0.000000,...,NaN,NaN,NaN,12,514.08,1750.56,171.84,0.0,0.0,230.6880
717558,35201127849963000110550010009990131009955543,2020-12-01,1403,275,935908,0,1,17,NaN,NaN,...,NaN,NaN,NaN,12,156.06,531.42,52.17,0.0,0.0,70.0308
717559,35201271605265000757590006717600920283957037,2020-12-06,5405,4,935908,1,1,1,NaN,NaN,...,4.119589,NaN,NaN,12,21.90,0.00,NaN,0.0,0.0,0.0000
717560,35201271605265000757590006717600921918979405,2020-12-07,5405,1,935908,1,1,1,NaN,NaN,...,4.119722,NaN,NaN,12,21.90,0.00,NaN,0.0,0.0,0.0000


In [7]:
tabela_2 = pd.read_csv(f'Loja_{loja}/loja_{loja}_tabela_2_v1.csv')
tabela_2 = tabela_2.sort_values(by=['COD_ITEM', 'DATA', 'IND_OPER', 'SUB_TIPO'], 
                           ascending=[True, True, True, False])
tabela_2

C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\2603824844.py:1: DtypeWarning: Columns (20,26) have mixed types. Specify dtype option on import or set low_memory=False.
  tabela_2 = pd.read_csv(f'Loja_{loja}/loja_{loja}_tabela_2_v1.csv')


,Unnamed: 0,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_EFD,...,Valor ICMS Operação,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO,Valor Base Cálculo ICMS Substituição Tributária,FONTE
599060,725302,35191071605265000757590005075110599441610811,2019-10-30,5405,1,18,1,1,1,NaN,...,0.0,6727 - DEPAKENE 250MG 25'S ABB C1,7891158000942.0,UN,30049099.0,NaN,NaN,NaN,NaN,dfe
19730,40146,35191027849963000110550010005997251005978274,2019-11-01,1403,2,18,0,1,1,1.0,...,0.0,DEPAKENE 250MG 25'S ABB C1,7891158000942,UN,30049099.0,1300100.0,2.784996e+13,7.160527e+13,NaN,entrada_sp
652542,795872,35191271605265000757590005075110698126997891,2019-12-11,5405,1,18,1,1,1,NaN,...,0.0,8471 - DEPAKENE 250MG 25S,7891158000942.0,UN,30049099.0,NaN,NaN,NaN,NaN,dfe
652543,795873,35191271605265000757590005075110698126997891,2019-12-11,5405,4,18,1,1,1,NaN,...,0.0,8471 - DEPAKENE 250MG 25S,7891158000942.0,UN,30049099.0,NaN,NaN,NaN,NaN,dfe
47582,71059,35191227849963000110550010006416671006396546,2019-12-13,1403,2,18,0,1,2,2.0,...,0.0,DEPAKENE 250MG 25'S ABB C1,7891158000942.0,UN,30049099.0,1300100.0,2.784996e+13,7.160527e+13,NaN,entrada_sp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191851,229687,35201127849963000110550010009983911009949320,2020-12-01,1403,1,935908,0,1,56,56.0,...,0.0,ATORVASTATINA 20MG 30S EMS,NaN,UN,30049099.0,NaN,2.784996e+13,7.160527e+13,NaN,entrada_sp
192238,230107,35201127849963000110550010009990131009955543,2020-12-01,1403,275,935908,0,1,17,17.0,...,0.0,ATORVASTATINA 20MG 30S EMS,NaN,UN,30049099.0,NaN,2.784996e+13,7.160527e+13,NaN,entrada_sp
639494,780707,35201271605265000757590006717600920283957037,2020-12-06,5405,4,935908,1,1,1,NaN,...,0.0,8199 - ATORVASTATINA 20MG 3,7896004729435.0,UN,30049099.0,NaN,NaN,NaN,NaN,dfe
639495,780708,35201271605265000757590006717600921918979405,2020-12-07,5405,1,935908,1,1,1,NaN,...,0.0,8199 - ATORVASTATINA 20MG 3,7896004729435.0,UN,30049099.0,NaN,NaN,NaN,NaN,dfe


In [8]:
planilha['DESCRICAO'] = tabela_2['DESCRICAO']
planilha['CODIGO_BARRA'] = tabela_2['CODIGO_BARRA']
planilha['UNIDADE'] = tabela_2['UNIDADE']
planilha['N C M'] = tabela_2['N C M']
planilha['CEST'] = tabela_2['CEST']
planilha['CNPJ EMITENTE'] = tabela_2['CNPJ EMITENTE']
planilha['CNPJ DESTINATARIO'] = tabela_2['CNPJ DESTINATARIO']

In [9]:
planilha.head()

,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,ICMS_INI,...,Valor ICMS Substituição Tributária,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO
0,35191071605265000757590005075110599441610811,2019-10-30,5405,1,18,1,1,1,2.0,10.192509,...,0.0,0.0,0.0000,Frusalt Pt 100g Cimed,SEM GTIN,UN,30044990.0,1300301.0,7.160527e+13,0.0
1,35191027849963000110550010005997251005978274,2019-11-01,1403,2,18,0,1,1,NaN,NaN,...,0.0,0.0,6.2424,Tylenol Sinus 500mg 24S,SEM GTIN,UN,30049045.0,1300101.0,7.160527e+13,0.0
2,35191271605265000757590005075110698126997891,2019-12-11,5405,1,18,1,1,1,NaN,NaN,...,0.0,0.0,0.0000,Selozok 25mg 30S Astrazeneca,SEM GTIN,UN,30049039.0,1300100.0,7.160527e+13,0.0
3,35191271605265000757590005075110698126997891,2019-12-11,5405,4,18,1,1,1,NaN,NaN,...,0.0,0.0,0.0000,Lrp Anthelios 70 Airli Clara,SEM GTIN,UN,33049990.0,2001600.0,7.160527e+13,0.0
4,35191227849963000110550010006416671006396546,2019-12-13,1403,2,18,0,1,2,NaN,NaN,...,0.0,0.0,12.5046,Perfenol 20S Uq,SEM GTIN,UN,30049036.0,1300301.0,7.160527e+13,0.0


In [10]:
planilha['DESCRICAO'] = planilha['DESCRICAO'].str.replace('"', "'")

In [11]:
planilha['N C M'] = pd.to_numeric(planilha['N C M'])
planilha['N C M'] = planilha['N C M'].astype(str).str.replace(r'\.0$', '', regex=True)

planilha['CEST'] = pd.to_numeric(planilha['CEST'])
planilha['CEST'] = planilha['CEST'].astype('Int64').astype(str).replace('<NA>', np.nan)
planilha['CEST'] = planilha['CEST'].astype(str).str.replace(r'\.0$', '', regex=True)

planilha['CNPJ EMITENTE'] = pd.to_numeric(planilha['CNPJ EMITENTE'])
planilha['CNPJ EMITENTE'] = planilha['CNPJ EMITENTE'].astype('Int64').astype(str).replace('<NA>', np.nan)

planilha['QTD_CAT'] = planilha['QTD_CAT'].astype(str) + ',000'

planilha['COD_LEGAL_PCAT'] = pd.to_numeric(planilha['COD_LEGAL_PCAT'])
planilha['COD_LEGAL_PCAT'] = planilha['COD_LEGAL_PCAT'].astype('Int64').astype(str).replace('<NA>', np.nan)

planilha['CFOP'] = pd.to_numeric(planilha['CFOP'])
planilha['CFOP'] = planilha['CFOP'].astype('Int64').astype(str).replace('<NA>', np.nan)

planilha['UNIDADE'] = planilha['UNIDADE'].str.strip()

def format_number(value):
    return '{:,.2f}'.format(value).replace(',', '.')

planilha

,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,ICMS_INI,...,Valor ICMS Substituição Tributária,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO
0,35191071605265000757590005075110599441610811,2019-10-30,5405,1,18,1,1,"1,000",2.0,10.192509,...,0.0,0.0,0.0000,Frusalt Pt 100g Cimed,SEM GTIN,UN,30044990,1300301,71605265000757,0.000000e+00
1,35191027849963000110550010005997251005978274,2019-11-01,1403,2,18,0,1,"1,000",NaN,NaN,...,0.0,0.0,6.2424,Tylenol Sinus 500mg 24S,SEM GTIN,UN,30049045,1300101,71605265000757,0.000000e+00
2,35191271605265000757590005075110698126997891,2019-12-11,5405,1,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0000,Selozok 25mg 30S Astrazeneca,SEM GTIN,UN,30049039,1300100,71605265000757,0.000000e+00
3,35191271605265000757590005075110698126997891,2019-12-11,5405,4,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0000,Lrp Anthelios 70 Airli Clara,SEM GTIN,UN,33049990,2001600,71605265000757,0.000000e+00
4,35191227849963000110550010006416671006396546,2019-12-13,1403,2,18,0,1,"2,000",NaN,NaN,...,0.0,0.0,12.5046,Perfenol 20S Uq,SEM GTIN,UN,30049036,1300301,71605265000757,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717557,35201127849963000110550010009983911009949320,2020-12-01,1403,1,935908,0,1,"56,000",0.0,0.000000,...,0.0,0.0,230.6880,TENADREN 40/25MG 30S SIGMA PH,7894916550116,UN,30049036,1300400,71605265000757,7.160527e+13
717558,35201127849963000110550010009990131009955543,2020-12-01,1403,275,935908,0,1,"17,000",NaN,NaN,...,0.0,0.0,70.0308,EBASTEL 10MG 10S EURO,7891317005351,UN,30049069,1300101,71605265000757,7.160527e+13
717559,35201271605265000757590006717600920283957037,2020-12-06,5405,4,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0000,RELVAR ELLIPTA 100/25MCG 30DS,7896015528829,UN,30049099,1300101,71605265000757,7.160527e+13
717560,35201271605265000757590006717600921918979405,2020-12-07,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0000,LEVOTIROXINA SOD 200MCG 30S M,7891721020247,UN,30043981,1300200,71605265000757,7.160527e+13


In [12]:
query = f'SELECT * FROM tabela_efds_modelo55 WHERE LOJA = {loja}'

efd = pd.read_sql(query, engine)
efd

,ID,LOJA,DATA_EFD,DATA_ENTRADA,DATA_SAIDA,CHAVE_ORIGINAL,COD_DO_ITEM,CODPROF,CFOP,DESC_PROD,QUANTI_EFDS,EAN,NCM,ICMS,CEST
0,8970579,100,09-2019,,,35190971605265000757550020000048561156824460,,,,,,,,,
1,8970580,100,09-2019,,,35190971605265000757550020000048831157444570,,,,,,,,,
2,8970581,100,09-2019,,,35190971605265000757550020000048791157442370,,,,,,,,,
3,8970582,100,09-2019,,,35190971605265000757550020000048801157442389,,,,,,,,,
4,8970583,100,09-2019,,,35190971605265000757550020000048811157442386,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247101,9217690,100,12-2020,05/12/2020,05/12/2020,35201271605265000757550020000060231183310549,1,87395,1411,DRENISON CR 30G BIOLAB SANUS,1,,30043290,18,13.003.01
247102,9217691,100,12-2020,05/12/2020,05/12/2020,35201271605265000757550020000060221183308873,1,222919,1411,Mam Fiona 240ml T2/Ros R801630,1,,39249000,18,20.063.00
247103,9217692,100,12-2020,05/12/2020,05/12/2020,35201271605265000757550020000060221183308873,2,222919,1411,Mam Fiona 240ml T2/Ros R801630,1,,39249000,18,20.063.00
247104,9217693,100,12-2020,07/12/2020,07/12/2020,35201271605265000757550020000060271183322597,1,204586,1411,Fr Bigfral Moviment F G/Xg C/8,1,,96190000,18,20.048.01


In [13]:
planilha = planilha.drop_duplicates()
planilha

,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,ICMS_INI,...,Valor ICMS Substituição Tributária,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO
0,35191071605265000757590005075110599441610811,2019-10-30,5405,1,18,1,1,"1,000",2.0,10.192509,...,0.0,0.0,0.0000,Frusalt Pt 100g Cimed,SEM GTIN,UN,30044990,1300301,71605265000757,0.000000e+00
1,35191027849963000110550010005997251005978274,2019-11-01,1403,2,18,0,1,"1,000",NaN,NaN,...,0.0,0.0,6.2424,Tylenol Sinus 500mg 24S,SEM GTIN,UN,30049045,1300101,71605265000757,0.000000e+00
2,35191271605265000757590005075110698126997891,2019-12-11,5405,1,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0000,Selozok 25mg 30S Astrazeneca,SEM GTIN,UN,30049039,1300100,71605265000757,0.000000e+00
3,35191271605265000757590005075110698126997891,2019-12-11,5405,4,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0000,Lrp Anthelios 70 Airli Clara,SEM GTIN,UN,33049990,2001600,71605265000757,0.000000e+00
4,35191227849963000110550010006416671006396546,2019-12-13,1403,2,18,0,1,"2,000",NaN,NaN,...,0.0,0.0,12.5046,Perfenol 20S Uq,SEM GTIN,UN,30049036,1300301,71605265000757,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717557,35201127849963000110550010009983911009949320,2020-12-01,1403,1,935908,0,1,"56,000",0.0,0.000000,...,0.0,0.0,230.6880,TENADREN 40/25MG 30S SIGMA PH,7894916550116,UN,30049036,1300400,71605265000757,7.160527e+13
717558,35201127849963000110550010009990131009955543,2020-12-01,1403,275,935908,0,1,"17,000",NaN,NaN,...,0.0,0.0,70.0308,EBASTEL 10MG 10S EURO,7891317005351,UN,30049069,1300101,71605265000757,7.160527e+13
717559,35201271605265000757590006717600920283957037,2020-12-06,5405,4,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0000,RELVAR ELLIPTA 100/25MCG 30DS,7896015528829,UN,30049099,1300101,71605265000757,7.160527e+13
717560,35201271605265000757590006717600921918979405,2020-12-07,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0000,LEVOTIROXINA SOD 200MCG 30S M,7891721020247,UN,30043981,1300200,71605265000757,7.160527e+13


In [14]:
planilha['CODIGO_BARRA'] = np.where(pd.notnull(planilha['CODIGO_BARRA']), planilha['CODIGO_BARRA'].astype(str).str.replace(r'\.0$', '', regex=True),
                                   np.nan)
planilha

,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,ICMS_INI,...,Valor ICMS Substituição Tributária,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO
0,35191071605265000757590005075110599441610811,2019-10-30,5405,1,18,1,1,"1,000",2.0,10.192509,...,0.0,0.0,0.0000,Frusalt Pt 100g Cimed,SEM GTIN,UN,30044990,1300301,71605265000757,0.000000e+00
1,35191027849963000110550010005997251005978274,2019-11-01,1403,2,18,0,1,"1,000",NaN,NaN,...,0.0,0.0,6.2424,Tylenol Sinus 500mg 24S,SEM GTIN,UN,30049045,1300101,71605265000757,0.000000e+00
2,35191271605265000757590005075110698126997891,2019-12-11,5405,1,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0000,Selozok 25mg 30S Astrazeneca,SEM GTIN,UN,30049039,1300100,71605265000757,0.000000e+00
3,35191271605265000757590005075110698126997891,2019-12-11,5405,4,18,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0000,Lrp Anthelios 70 Airli Clara,SEM GTIN,UN,33049990,2001600,71605265000757,0.000000e+00
4,35191227849963000110550010006416671006396546,2019-12-13,1403,2,18,0,1,"2,000",NaN,NaN,...,0.0,0.0,12.5046,Perfenol 20S Uq,SEM GTIN,UN,30049036,1300301,71605265000757,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717557,35201127849963000110550010009983911009949320,2020-12-01,1403,1,935908,0,1,"56,000",0.0,0.000000,...,0.0,0.0,230.6880,TENADREN 40/25MG 30S SIGMA PH,7894916550116,UN,30049036,1300400,71605265000757,7.160527e+13
717558,35201127849963000110550010009990131009955543,2020-12-01,1403,275,935908,0,1,"17,000",NaN,NaN,...,0.0,0.0,70.0308,EBASTEL 10MG 10S EURO,7891317005351,UN,30049069,1300101,71605265000757,7.160527e+13
717559,35201271605265000757590006717600920283957037,2020-12-06,5405,4,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0000,RELVAR ELLIPTA 100/25MCG 30DS,7896015528829,UN,30049099,1300101,71605265000757,7.160527e+13
717560,35201271605265000757590006717600921918979405,2020-12-07,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0,0.0000,LEVOTIROXINA SOD 200MCG 30S M,7891721020247,UN,30043981,1300200,71605265000757,7.160527e+13


In [15]:
mes_ano = []
for date in planilha['DATA']:
    mes_ano.append(str(date)[5:7] + str(date)[0:4])

planilha['Data'] = mes_ano
planilha['Data'].unique()

array(['102019', '112019', '122019', '072020', '092020', '112020',
       '122020', '012020', '022020', '042020', '102020', '052020',
       '062020', '032020', '082020'], dtype=object)

In [16]:
planilha.replace('nan', np.nan, inplace=True)

In [17]:
planilha['DATA'].astype(str).iloc[0][8:]

'30'

In [18]:
planilha

,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,ICMS_INI,...,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO,Data
0,35191071605265000757590005075110599441610811,2019-10-30,5405,1,18,1,1,"1,000",2.0,10.192509,...,0.0,0.0000,Frusalt Pt 100g Cimed,SEM GTIN,UN,30044990,1300301,71605265000757,0.000000e+00,102019
1,35191027849963000110550010005997251005978274,2019-11-01,1403,2,18,0,1,"1,000",NaN,NaN,...,0.0,6.2424,Tylenol Sinus 500mg 24S,SEM GTIN,UN,30049045,1300101,71605265000757,0.000000e+00,112019
2,35191271605265000757590005075110698126997891,2019-12-11,5405,1,18,1,1,"1,000",NaN,NaN,...,0.0,0.0000,Selozok 25mg 30S Astrazeneca,SEM GTIN,UN,30049039,1300100,71605265000757,0.000000e+00,122019
3,35191271605265000757590005075110698126997891,2019-12-11,5405,4,18,1,1,"1,000",NaN,NaN,...,0.0,0.0000,Lrp Anthelios 70 Airli Clara,SEM GTIN,UN,33049990,2001600,71605265000757,0.000000e+00,122019
4,35191227849963000110550010006416671006396546,2019-12-13,1403,2,18,0,1,"2,000",NaN,NaN,...,0.0,12.5046,Perfenol 20S Uq,SEM GTIN,UN,30049036,1300301,71605265000757,0.000000e+00,122019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717557,35201127849963000110550010009983911009949320,2020-12-01,1403,1,935908,0,1,"56,000",0.0,0.000000,...,0.0,230.6880,TENADREN 40/25MG 30S SIGMA PH,7894916550116,UN,30049036,1300400,71605265000757,7.160527e+13,122020
717558,35201127849963000110550010009990131009955543,2020-12-01,1403,275,935908,0,1,"17,000",NaN,NaN,...,0.0,70.0308,EBASTEL 10MG 10S EURO,7891317005351,UN,30049069,1300101,71605265000757,7.160527e+13,122020
717559,35201271605265000757590006717600920283957037,2020-12-06,5405,4,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0000,RELVAR ELLIPTA 100/25MCG 30DS,7896015528829,UN,30049099,1300101,71605265000757,7.160527e+13,122020
717560,35201271605265000757590006717600921918979405,2020-12-07,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0000,LEVOTIROXINA SOD 200MCG 30S M,7891721020247,UN,30043981,1300200,71605265000757,7.160527e+13,122020


In [19]:
planilha['CEST'] = planilha['CEST'].replace('0', pd.NA)

# Second, add leading '0' to values with length equal to 6
planilha.loc[planilha['CEST'].str.len() == 6, 'CEST'] = '0' + planilha['CEST']

planilha['CEST']

0         1300301
1         1300101
2         1300100
3         2001600
4         1300301
           ...   
717557    1300400
717558    1300101
717559    1300101
717560    1300200
717561    2001400
Name: CEST, Length: 717562, dtype: object

In [20]:
planilha['N C M'] = planilha['N C M'].apply(lambda x: str(x).zfill(8) if len(str(x)) < 8 else str(x))
planilha

,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,ICMS_INI,...,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO,Data
0,35191071605265000757590005075110599441610811,2019-10-30,5405,1,18,1,1,"1,000",2.0,10.192509,...,0.0,0.0000,Frusalt Pt 100g Cimed,SEM GTIN,UN,30044990,1300301,71605265000757,0.000000e+00,102019
1,35191027849963000110550010005997251005978274,2019-11-01,1403,2,18,0,1,"1,000",NaN,NaN,...,0.0,6.2424,Tylenol Sinus 500mg 24S,SEM GTIN,UN,30049045,1300101,71605265000757,0.000000e+00,112019
2,35191271605265000757590005075110698126997891,2019-12-11,5405,1,18,1,1,"1,000",NaN,NaN,...,0.0,0.0000,Selozok 25mg 30S Astrazeneca,SEM GTIN,UN,30049039,1300100,71605265000757,0.000000e+00,122019
3,35191271605265000757590005075110698126997891,2019-12-11,5405,4,18,1,1,"1,000",NaN,NaN,...,0.0,0.0000,Lrp Anthelios 70 Airli Clara,SEM GTIN,UN,33049990,2001600,71605265000757,0.000000e+00,122019
4,35191227849963000110550010006416671006396546,2019-12-13,1403,2,18,0,1,"2,000",NaN,NaN,...,0.0,12.5046,Perfenol 20S Uq,SEM GTIN,UN,30049036,1300301,71605265000757,0.000000e+00,122019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717557,35201127849963000110550010009983911009949320,2020-12-01,1403,1,935908,0,1,"56,000",0.0,0.000000,...,0.0,230.6880,TENADREN 40/25MG 30S SIGMA PH,7894916550116,UN,30049036,1300400,71605265000757,7.160527e+13,122020
717558,35201127849963000110550010009990131009955543,2020-12-01,1403,275,935908,0,1,"17,000",NaN,NaN,...,0.0,70.0308,EBASTEL 10MG 10S EURO,7891317005351,UN,30049069,1300101,71605265000757,7.160527e+13,122020
717559,35201271605265000757590006717600920283957037,2020-12-06,5405,4,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0000,RELVAR ELLIPTA 100/25MCG 30DS,7896015528829,UN,30049099,1300101,71605265000757,7.160527e+13,122020
717560,35201271605265000757590006717600921918979405,2020-12-07,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0000,LEVOTIROXINA SOD 200MCG 30S M,7891721020247,UN,30043981,1300200,71605265000757,7.160527e+13,122020


In [21]:
planilha.columns

Index(['CHV_DOC', 'DATA', 'CFOP', 'NUM_ITEM', 'COD_ITEM', 'IND_OPER',
       'SUB_TIPO', 'QTD_CAT', 'QTD_INI', 'ICMS_INI', 'QTD_ent1_devolv_ent',
       'ICMS_TOT_0', 'ICMS_TOT_ent_unit', 'ULT_ICMS_TOT_ent_unit',
       'ICMS_TOT_1', 'qtd_saida_1_devolv_saida', 'ICMS_SAIDA_UNI',
       'ULT_ICMS_SAIDA_UNI', 'ICMS_SAIDA', 'ICMS_TOT_PCAT', 'VL_CONFR_0',
       'VLR_CONFR_UNIT', 'ULT_VLR_CONFR_UNIT', 'VLR_CONFR_1', 'QTD_SALDO',
       'ICMS_TOT_SALDO', 'VLR_RESSARCIMENTO', 'VLR_COMPLEMENTO', 'COD_LEGAL',
       'COD_LEGAL_PCAT', 'VLR_CONFR_PCAT', 'SALDO_FINAL_MES_QTD',
       'SALDO_FINAL_MES_ICMS', 'ALIQUOTA', 'VALOR',
       'Valor Base Cálculo ICMS ST Retido Operação Anterior',
       'Valor Complementar', 'Valor ICMS Substituição Tributária',
       'Valor ICMS Operação', 'ICMS_TOT', 'DESCRICAO', 'CODIGO_BARRA',
       'UNIDADE', 'N C M', 'CEST', 'CNPJ EMITENTE', 'CNPJ DESTINATARIO',
       'Data'],
      dtype='object')

In [22]:
planilha[planilha['CODIGO_BARRA'] == 'SEM GTIN      ']

,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,ICMS_INI,...,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO,Data
0,35191071605265000757590005075110599441610811,2019-10-30,5405,1,18,1,1,"1,000",2.0,10.192509,...,0.0,0.0000,Frusalt Pt 100g Cimed,SEM GTIN,UN,30044990,1300301,71605265000757,0.000000e+00,102019
1,35191027849963000110550010005997251005978274,2019-11-01,1403,2,18,0,1,"1,000",NaN,NaN,...,0.0,6.2424,Tylenol Sinus 500mg 24S,SEM GTIN,UN,30049045,1300101,71605265000757,0.000000e+00,112019
2,35191271605265000757590005075110698126997891,2019-12-11,5405,1,18,1,1,"1,000",NaN,NaN,...,0.0,0.0000,Selozok 25mg 30S Astrazeneca,SEM GTIN,UN,30049039,1300100,71605265000757,0.000000e+00,122019
3,35191271605265000757590005075110698126997891,2019-12-11,5405,4,18,1,1,"1,000",NaN,NaN,...,0.0,0.0000,Lrp Anthelios 70 Airli Clara,SEM GTIN,UN,33049990,2001600,71605265000757,0.000000e+00,122019
4,35191227849963000110550010006416671006396546,2019-12-13,1403,2,18,0,1,"2,000",NaN,NaN,...,0.0,12.5046,Perfenol 20S Uq,SEM GTIN,UN,30049036,1300301,71605265000757,0.000000e+00,122019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716535,35200171605265000757590005075110787697682430,2020-01-15,5405,3,932244,1,1,"1,000",2.0,24.311182,...,0.0,0.0000,Des Nivea R M 50ml Fresh Activ,SEM GTIN,UN,33072010,2801600,71605265000757,7.160527e+13,012020
716668,35190944463156000184550820002813791106344680,2019-10-01,1403,5,932814,0,1,"1,000",3.0,15.222727,...,0.0,6.6575,"Protopic 0,3mg Pda 10g Astella",SEM GTIN,UN,30049078,1300100,71605265000757,7.160527e+13,102019
716980,35200927849963000110550010009130751009100010,2020-09-11,1403,159,933184,0,1,"3,000",NaN,NaN,...,0.0,14.9160,"Protopic 0,3mg Pda 10g",SEM GTIN,UN,30049078,1300100,71605265000757,7.160527e+13,092020
717017,35201271605265000757590005075110910232659870,2020-12-22,5405,4,933184,1,1,"1,000",NaN,NaN,...,0.0,0.0000,Nivea Prot Labial Repair,SEM GTIN,UN,33049990,<NA>,71605265000757,7.160527e+13,122020


In [23]:
planilha['CODIGO_BARRA'] = planilha['CODIGO_BARRA'].replace('SEM GTIN      ', np.nan)
planilha

,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,ICMS_INI,...,Valor ICMS Operação,ICMS_TOT,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO,Data
0,35191071605265000757590005075110599441610811,2019-10-30,5405,1,18,1,1,"1,000",2.0,10.192509,...,0.0,0.0000,Frusalt Pt 100g Cimed,NaN,UN,30044990,1300301,71605265000757,0.000000e+00,102019
1,35191027849963000110550010005997251005978274,2019-11-01,1403,2,18,0,1,"1,000",NaN,NaN,...,0.0,6.2424,Tylenol Sinus 500mg 24S,NaN,UN,30049045,1300101,71605265000757,0.000000e+00,112019
2,35191271605265000757590005075110698126997891,2019-12-11,5405,1,18,1,1,"1,000",NaN,NaN,...,0.0,0.0000,Selozok 25mg 30S Astrazeneca,NaN,UN,30049039,1300100,71605265000757,0.000000e+00,122019
3,35191271605265000757590005075110698126997891,2019-12-11,5405,4,18,1,1,"1,000",NaN,NaN,...,0.0,0.0000,Lrp Anthelios 70 Airli Clara,NaN,UN,33049990,2001600,71605265000757,0.000000e+00,122019
4,35191227849963000110550010006416671006396546,2019-12-13,1403,2,18,0,1,"2,000",NaN,NaN,...,0.0,12.5046,Perfenol 20S Uq,NaN,UN,30049036,1300301,71605265000757,0.000000e+00,122019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717557,35201127849963000110550010009983911009949320,2020-12-01,1403,1,935908,0,1,"56,000",0.0,0.000000,...,0.0,230.6880,TENADREN 40/25MG 30S SIGMA PH,7894916550116,UN,30049036,1300400,71605265000757,7.160527e+13,122020
717558,35201127849963000110550010009990131009955543,2020-12-01,1403,275,935908,0,1,"17,000",NaN,NaN,...,0.0,70.0308,EBASTEL 10MG 10S EURO,7891317005351,UN,30049069,1300101,71605265000757,7.160527e+13,122020
717559,35201271605265000757590006717600920283957037,2020-12-06,5405,4,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0000,RELVAR ELLIPTA 100/25MCG 30DS,7896015528829,UN,30049099,1300101,71605265000757,7.160527e+13,122020
717560,35201271605265000757590006717600921918979405,2020-12-07,5405,1,935908,1,1,"1,000",NaN,NaN,...,0.0,0.0000,LEVOTIROXINA SOD 200MCG 30S M,7891721020247,UN,30043981,1300200,71605265000757,7.160527e+13,122020


In [24]:
list(planilha['CODIGO_BARRA'].unique())

[nan,
 '7896269900983 ',
 '7896019602006 ',
 '7896261000223 ',
 '7898444418276 ',
 '7896206407384 ',
 '7896015518318 ',
 '7899095201187 ',
 '7896523226972 ',
 '7899095246676 ',
 '7891106909624 ',
 '7894916500296 ',
 '7896637022811 ',
 '7897322700724 ',
 '7896004706795 ',
 '7896714214115 ',
 '7896422500111 ',
 '7896181921325 ',
 '7899095238299 ',
 '7891142115492',
 '7891142141293',
 '7891142130822',
 '7897322709420',
 '7896094903067',
 '7896094901148',
 '7897322701998',
 '7896179850026',
 '7891106904155',
 '7896261000216',
 '7896269900211',
 '7896026300223',
 '7896658003523',
 '7896658003141',
 '7896181908890',
 '7896548111727',
 '7891106906029',
 '7896016801655',
 '7898916010052',
 '7896269900334',
 '7896637023665',
 '7896261010055',
 '7891058059149',
 '7898074618183',
 '5702191007985',
 '7894916509305',
 '7891721010026',
 '7896422507547',
 '7891106906012',
 '7891317465650',
 '7896658000713',
 '7896658002700',
 '7897595900029',
 '7890282800015',
 '7891317478919',
 '7896658010286',
 '78

In [25]:
# Geração do dataset base para geração da 1100

cols_1100 = ['CHV_DOC', 'DATA', 'NUM_ITEM', 'IND_OPER', 'COD_ITEM', 'CFOP','QTD_CAT', 'ICMS_TOT_PCAT', 'VLR_CONFR_PCAT', 'COD_LEGAL_PCAT']

df_1100 = planilha.copy()
df_1100 = df_1100[cols_1100]
df_1100['Data'] = df_1100['DATA']
df_1100['DATA'] = df_1100['DATA'].astype(str)
df_1100['DATA'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%d%m%Y') for x in df_1100['DATA']]
#df_1100.rename(coluns={'Número Item': 'NUM_ITEM'}, inplace=True)


df_1100['QTD_FIN'] = df_1100['QTD_CAT']
df_1100['COD_REG'] = ['1100']*df_1100.shape[0]
df_1100 = df_1100[['COD_REG','CHV_DOC','Data','DATA','NUM_ITEM','IND_OPER','COD_ITEM','CFOP','QTD_FIN','ICMS_TOT_PCAT','VLR_CONFR_PCAT','COD_LEGAL_PCAT']]
df_1100

,COD_REG,CHV_DOC,Data,DATA,NUM_ITEM,IND_OPER,COD_ITEM,CFOP,QTD_FIN,ICMS_TOT_PCAT,VLR_CONFR_PCAT,COD_LEGAL_PCAT
0,1100,35191071605265000757590005075110599441610811,2019-10-30,30102019,1,1,18,5405,"1,000",NaN,5.096254,1
1,1100,35191027849963000110550010005997251005978274,2019-11-01,01112019,2,0,18,1403,"1,000",6.2424,NaN,NaN
2,1100,35191271605265000757590005075110698126997891,2019-12-11,11122019,1,1,18,5405,"1,000",NaN,5.680000,1
3,1100,35191271605265000757590005075110698126997891,2019-12-11,11122019,4,1,18,5405,"1,000",NaN,5.680000,1
4,1100,35191227849963000110550010006416671006396546,2019-12-13,13122019,2,0,18,1403,"2,000",12.5046,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
717557,1100,35201127849963000110550010009983911009949320,2020-12-01,01122020,1,0,935908,1403,"56,000",230.6880,NaN,NaN
717558,1100,35201127849963000110550010009990131009955543,2020-12-01,01122020,275,0,935908,1403,"17,000",70.0308,NaN,NaN
717559,1100,35201271605265000757590006717600920283957037,2020-12-06,06122020,4,1,935908,5405,"1,000",NaN,4.119589,1
717560,1100,35201271605265000757590006717600921918979405,2020-12-07,07122020,1,1,935908,5405,"1,000",NaN,4.119722,1


In [26]:
df_1100['Data'] = df_1100['Data'].astype(str)
df_1100['Data'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%m%Y') for x in df_1100['Data']]
df_1100['Data']

0         102019
1         112019
2         122019
3         122019
4         122019
           ...  
717557    122020
717558    122020
717559    122020
717560    122020
717561    122020
Name: Data, Length: 717562, dtype: object

In [27]:
df_1100.dtypes

COD_REG            object
CHV_DOC            object
Data               object
DATA               object
NUM_ITEM            int64
IND_OPER            int64
COD_ITEM            int64
CFOP               object
QTD_FIN            object
ICMS_TOT_PCAT     float64
VLR_CONFR_PCAT    float64
COD_LEGAL_PCAT     object
dtype: object

In [28]:
df_1100['QTD_FIN'] = df_1100['QTD_FIN'].replace('nan', np.nan)
df_1100

,COD_REG,CHV_DOC,Data,DATA,NUM_ITEM,IND_OPER,COD_ITEM,CFOP,QTD_FIN,ICMS_TOT_PCAT,VLR_CONFR_PCAT,COD_LEGAL_PCAT
0,1100,35191071605265000757590005075110599441610811,102019,30102019,1,1,18,5405,"1,000",NaN,5.096254,1
1,1100,35191027849963000110550010005997251005978274,112019,01112019,2,0,18,1403,"1,000",6.2424,NaN,NaN
2,1100,35191271605265000757590005075110698126997891,122019,11122019,1,1,18,5405,"1,000",NaN,5.680000,1
3,1100,35191271605265000757590005075110698126997891,122019,11122019,4,1,18,5405,"1,000",NaN,5.680000,1
4,1100,35191227849963000110550010006416671006396546,122019,13122019,2,0,18,1403,"2,000",12.5046,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
717557,1100,35201127849963000110550010009983911009949320,122020,01122020,1,0,935908,1403,"56,000",230.6880,NaN,NaN
717558,1100,35201127849963000110550010009990131009955543,122020,01122020,275,0,935908,1403,"17,000",70.0308,NaN,NaN
717559,1100,35201271605265000757590006717600920283957037,122020,06122020,4,1,935908,5405,"1,000",NaN,4.119589,1
717560,1100,35201271605265000757590006717600921918979405,122020,07122020,1,1,935908,5405,"1,000",NaN,4.119722,1


In [29]:
# Leitura do arquivo da 1050

arquivo_1050 = pd.read_excel(f'Loja_{loja}/1050_loja_{loja}_v1.xlsx', dtype=str)

In [30]:
mes_ano = []
for date in arquivo_1050['REF']:
    mes_ano.append(str(date)[5:7] + str(date)[0:4])
    
arquivo_1050['DATA'] = [datetime.strptime(x, '%Y%m').strftime('%m%Y') for x in arquivo_1050['REF']]
arquivo_1050

,Unnamed: 0,REF,COD_ITEM,QTD_INI,ICMS_INI,SALDO_FINAL_MES_QTD,SALDO_FINAL_MES_ICMS,DATA
0,0,201910,18,2,10.19250864652384,1,5.109999999999999,102019
1,1,201911,18,1,5.109999999999999,2,11.36,112019
2,2,201912,18,2,11.36,2,12.51,122019
3,3,202001,18,2,12.51,2,12.51,012020
4,4,202002,18,2,12.51,2,12.51,022020
...,...,...,...,...,...,...,...,...
101065,101065,202008,935908,0,0,0,0,082020
101066,101066,202009,935908,0,0,0,0,092020
101067,101067,202010,935908,0,0,0,0,102020
101068,101068,202011,935908,0,0,0,0,112020


In [31]:
for date in planilha['Data'].unique():
    print(date)

102019
112019
122019
072020
092020
112020
122020
012020
022020
042020
102020
052020
062020
032020
082020


In [32]:
# Definição de uma função para exportar os
# dataframes de cada documentação em um
# arquivo .txt com a formatação desejada

def dataframe_to_txt_with_crlf(data, path):
    with open(path, 'a', newline='', encoding='utf-8') as f:
        for index, row in data.iterrows():
            row_str = '|'.join(map(lambda x: '' if pd.isna(x) else str(x), row)) + '\r\n'
            f.write(row_str)

In [33]:
def format_number(value):
    locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')  # Set your desired locale
    formatted_value = locale.format('%.2f', value, grouping=True)
    formatted_value = str(formatted_value).replace('.', '')
    return formatted_value

In [40]:
df_0150 = pd.read_excel('registros_0000_0150.xlsx', sheet_name='0150', dtype=str)
df_0150 = df_0150[['LOJA', 'REG', 'COD_PART', 'NOME', 'COD_PAIS', 'CNPJ', 'CPF', 'COD_MUN', 'IE', 'DATA']]
df_0150

,LOJA,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,DATA
0,Loja 138,0150,FOR753006975,DIS COMERCIO DE ELTRODOMESTICOS LTDA,1058,23209013000332,NaN,3205002,105535836117,20201201
1,Loja 211,0150,FOR752988443,COMERCIO DE MATERIAIS PARA CONSTRUCOES PATRICI...,1058,61253282000123,NaN,3550308,109150975113,20200801
2,Loja 188,0150,CLI752000293,Associacao dos advogados de sao paulo,1058,62500855000139,NaN,3550308,112051770110,20200701
3,Loja 192,0150,FOR753008845,ELETRICA COML ANDRA LT,1058,47674429000390,NaN,3550308,112372210111,20201201
4,Loja 44,0150,FOR753005939,ASSOCIAÇÃO DOS FUNCIONARIOS PUBLICOS DO ESTADO...,1058,62149000000105,NaN,3550308,112669417110,20201101
...,...,...,...,...,...,...,...,...,...,...
34274,Loja 71,0150,FOR753007082,1606791600000,1058,NaN,30595271804,3550308,NaN,20201201
34275,Loja 71,0150,FOR753007140,valdice gouveia,1058,NaN,00149812892,3550308,NaN,20201201
34276,Loja 71,0150,FOR753007775,ANA MARIA LANDOLPHO BAPTISTA,1058,NaN,17435905801,3550308,NaN,20201201
34277,Loja 71,0150,FOR753009445,CLEONICE MOREIRA DA SILVA,1058,NaN,20781164591,3550308,NaN,20201201


In [41]:
df_0150 = df_0150[df_0150['LOJA'] == f'Loja {loja}']
df_0150['DATA'] = df_0150['DATA'].str.slice(0, 10).astype(str)
df_0150['DATA'] = [datetime.strptime(x, '%Y%m%d').strftime('%m%Y') for x in df_0150['DATA']]
df_0150['CNPJ'] = df_0150['CNPJ'].replace('', np.nan)
df_0150['IE'] = df_0150['IE'].replace('', np.nan)
df_0150['COD_PART'] = df_0150['COD_PART'].str[3:]
df_0150

C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\1763487891.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_0150['DATA'] = df_0150['DATA'].str.slice(0, 10).astype(str)
C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\1763487891.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_0150['DATA'] = [datetime.strptime(x, '%Y%m%d').strftime('%m%Y') for x in df_0150['DATA']]
C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\1763487891.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy 

,LOJA,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,COD_MUN,IE,DATA
318,Loja 100,0150,751895505,GILDERSON AUGUSTO FONSECA EIRELI ME,1058,29028142000149,NaN,3550308,118720948110,092020
319,Loja 100,0150,751895505,GILDERSON AUGUSTO FONSECA EIRELI ME,1058,29028142000149,NaN,3550308,118720948110,102020
327,Loja 100,0150,000000470,Master Formula Farmacia de Manipulacao LTDA,1058,71605265002024,NaN,3550308,119507430112,092019
328,Loja 100,0150,000000470,Master Formula Farmacia de Manipulacao LTDA,1058,71605265002024,NaN,3550308,119507430112,092019
329,Loja 100,0150,000000470,Master Formula Farmacia de Manipulacao LTDA,1058,71605265002024,NaN,3550308,119507430112,102019
...,...,...,...,...,...,...,...,...,...,...
15440,Loja 100,0150,753012839,JOAO ANTONIO DO NASCIMENTO,1058,NaN,09820140820,3550308,NaN,122020
15441,Loja 100,0150,753013242,SIMONE SOARES DA SILVA,1058,NaN,12711754871,3550308,NaN,122020
15442,Loja 100,0150,753013527,ANDREIA MOURA DOS SANTOS,1058,NaN,32509444832,3550308,NaN,122020
15443,Loja 100,0150,753013819,TERUAKI HONDA,1058,NaN,10740329855,3550308,NaN,122020


In [42]:
df_0150 = df_0150[['REG', 'COD_PART', 'NOME', 'COD_PAIS', 'CNPJ', 'CPF', 'IE', 'COD_MUN', 'DATA']]
df_0150 = df_0150[df_0150['CNPJ'].notnull()]
df_0150 = df_0150.drop_duplicates()
df_0150

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
318,0150,751895505,GILDERSON AUGUSTO FONSECA EIRELI ME,1058,29028142000149,NaN,118720948110,3550308,092020
319,0150,751895505,GILDERSON AUGUSTO FONSECA EIRELI ME,1058,29028142000149,NaN,118720948110,3550308,102020
327,0150,000000470,Master Formula Farmacia de Manipulacao LTDA,1058,71605265002024,NaN,119507430112,3550308,092019
329,0150,000000470,Master Formula Farmacia de Manipulacao LTDA,1058,71605265002024,NaN,119507430112,3550308,102019
331,0150,000000470,Master Formula Farmacia de Manipulacao LTDA,1058,71605265002024,NaN,119507430112,3550308,112019
...,...,...,...,...,...,...,...,...,...
15177,0150,000329865,Consumidor final,1058,96291141014300,NaN,NaN,3550308,052020
15178,0150,000002486,Penitenciaria Feminina Sant Ana,1058,96291141014300,NaN,NaN,3550308,052020
15277,0150,751975901,Fundacao abc,1058,57571275001921,NaN,NaN,3550308,082020
15285,0150,752006043,M2 garcia materiais de construcao ltda,1058,09292174000140,NaN,NaN,3550308,082020


In [43]:
mask_duplicates = df_0150[df_0150['DATA'] == '112019'].duplicated(subset='CNPJ', keep='first')
# Invert the mask to keep the first occurrence of each unique 'CNPJ' value
mask_to_keep = ~mask_duplicates

# Apply the mask to the DataFrame to keep the desired rows for the current 'PERIODO'
filtered_df = df_0150[(df_0150['DATA'] == '112019') | mask_to_keep]
filtered_df

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
331,0150,000000470,Master Formula Farmacia de Manipulacao LTDA,1058,71605265002024,NaN,119507430112,3550308,112019
709,0150,000355698,Conde atacadista LTDA,1058,27849963000110,NaN,125415766116,3508504,112019
2053,0150,000000496,Master Fórmula Farmácia de Manipulação LTDA,1058,71605265004663,NaN,126790472110,3550308,112019
2137,0150,000000497,Master Fórmula Farmácia de Manipulação LTDA,1058,71605265004744,NaN,127036753117,3518800,112019
2397,0150,751972176,Eletropaulo Metropolitana Eletricidade de São ...,1058,61695227000193,NaN,133122090117,3505708,112019
2524,0150,000000457,Master formula farmácia de manipulação ltda- me,1058,71605265000757,NaN,143351809116,3550308,112019
2525,0150,000000457,Master Fórmula Farmácia de Manipulação Ltda - ME,1058,71605265000757,NaN,143351809116,3550308,112019
2909,0150,000002016,Servimed comercial ltda,1058,44463156000184,NaN,209017636112,3506003,112019
3876,0150,000000495,Master fórmula farmacia de manipulação LTDA,1058,71605265004582,NaN,254233469115,3510500,112019
4433,0150,752007028,Dbr distrib brasileira de recargas ltda,1058,07259917000154,NaN,28334171-8,5002704,112019


In [44]:
filtered_df['CNPJ'].unique()

array(['71605265002024', '27849963000110', '71605265004663',
       '71605265004744', '61695227000193', '71605265000757',
       '44463156000184', '71605265004582', '07259917000154',
       '01206820000520', '61940292000137', '05651966000455',
       '71605265000404', '45453214001042', '97554090000102',
       '10328983000143', '23276873000127', '61068276001682',
       '71605265000838', '71605265000676', '96291141014300'], dtype=object)

In [45]:
df_0150_final = pd.DataFrame()

# Get the unique 'PERIODO' values
unique_periodos = df_0150['DATA'].unique()

# Iterate through unique 'PERIODO' values
for periodo in unique_periodos:
    # Filter the DataFrame for the current 'PERIODO' value
    filtered_df = df_0150[df_0150['DATA'] == periodo]
    
    # Drop duplicates in the 'CNPJ' column for the current 'PERIODO'
    filtered_df = filtered_df.drop_duplicates(subset='CNPJ', keep='first')
    
    # Append the filtered results to the final DataFrame
    df_0150_final = df_0150_final.append(filtered_df, ignore_index=True)

# Reset the index of the final DataFrame
df_0150_final.reset_index(drop=True, inplace=True)
df_0150_final = df_0150_final[df_0150_final['REG'] == '0150']
df_0150_final

C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\575281542.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_0150_final = df_0150_final.append(filtered_df, ignore_index=True)


,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,751895505,GILDERSON AUGUSTO FONSECA EIRELI ME,1058,29028142000149,NaN,118720948110,3550308,092020
1,0150,000000470,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265002024,NaN,119507430112,3550308,092020
2,0150,000355698,CONDE ATACADISTA LTDA,1058,27849963000110,NaN,125415766116,3508504,092020
3,0150,000000499,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265004906,NaN,128210260116,3550308,092020
4,0150,000001604,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265005554,NaN,128970130118,3550308,092020
...,...,...,...,...,...,...,...,...,...
357,0150,000002004,DHV DISTRIBUIDORA HOSPITALAR DO VALE LTDA,1058,23276873000127,NaN,645739349118,3549904,122020
358,0150,000000458,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000838,NaN,796260079110,3518800,122020
359,0150,000000456,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000676,NaN,796436256113,3518800,122020
360,0150,752988273,B.A. SILVA ELETRONICOS E MANUTENCAO LTDA,1058,37636697000100,NaN,392350441112,3524402,122020


In [46]:
df_0150_final = df_0150_final.drop_duplicates()
df_0150_final[df_0150_final['IE'].notnull()]

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,751895505,GILDERSON AUGUSTO FONSECA EIRELI ME,1058,29028142000149,NaN,118720948110,3550308,092020
1,0150,000000470,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265002024,NaN,119507430112,3550308,092020
2,0150,000355698,CONDE ATACADISTA LTDA,1058,27849963000110,NaN,125415766116,3508504,092020
3,0150,000000499,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265004906,NaN,128210260116,3550308,092020
4,0150,000001604,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265005554,NaN,128970130118,3550308,092020
...,...,...,...,...,...,...,...,...,...
356,0150,000007953,JULIA COSMETICOS EIRELI,1058,10328983000143,NaN,645538983114,3549904,122020
357,0150,000002004,DHV DISTRIBUIDORA HOSPITALAR DO VALE LTDA,1058,23276873000127,NaN,645739349118,3549904,122020
358,0150,000000458,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000838,NaN,796260079110,3518800,122020
359,0150,000000456,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000676,NaN,796436256113,3518800,122020


In [47]:
df_cnpj_repetido = pd.DataFrame(columns=df_0150_final.columns)

dates = ['102019', '112019', '122019',
        '012020', '022020', '032020', '042020',
        '052020', '062020', '072020', '082020',
        '092020', '102020', '112020', '122020']
for date in dates:
    for cnpj in df_0150_final[df_0150_final['DATA'] == date]['CNPJ'].unique():
        if len(df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['CNPJ'] == cnpj)]) > 1:
            df = df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['CNPJ'] == cnpj)]
            df_cnpj_repetido = pd.concat([df_cnpj_repetido, df])
            
df_cnpj_repetido

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA


In [48]:
df_ie_repetido = pd.DataFrame(columns=df_0150_final.columns)

dates = ['102019', '112019', '122019',
        '012020', '022020', '032020', '042020',
        '052020', '062020', '072020', '082020',
        '092020', '102020', '112020', '122020']
for date in dates:
    for ie in df_0150_final[df_0150_final['DATA'] == date]['IE'].unique():
        if len(df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['IE'] == ie)]) > 1:
            df = df_0150_final.loc[(df_0150_final['DATA'] == date) & (df_0150_final['IE'] == ie)]
            df_ie_repetido = pd.concat([df_ie_repetido, df])
            
df_ie_repetido

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA


In [79]:
# Iterate through each file in the folder
df_0000 = pd.read_excel('registros_0000_0150.xlsx', sheet_name='0000', dtype=str)

df_0000['DATA'] = df_0000['DATA'].astype(str).str.slice(0, 8)
df_0000['DATA'] = [datetime.strptime(x, '%Y%m%d').strftime('%m%Y') for x in df_0000['DATA']]
df_0000['COD_VER'] = ['01'] * df_0000.shape[0]
df_0000['COD_FIN'] = ['00'] * df_0000.shape[0]
df_0000 = df_0000[['LOJA','REG', 'DATA', 'NOME', 'CNPJ', 'IE',
        'COD_MUN', 'COD_VER', 'COD_FIN']]

df_0000

,LOJA,REG,DATA,NOME,CNPJ,IE,COD_MUN,COD_VER,COD_FIN
0,Loja 100,0000,092019,Master Formula Farmacia de Manipulacao LTDA,71605265000757,143351809116,3550308,01,00
1,Loja 100,0000,102019,Master Formula Farmacia de Manipulacao LTDA,71605265000757,143351809116,3550308,01,00
2,Loja 100,0000,112019,Master Formula Farmacia de Manipulacao LTDA,71605265000757,143351809116,3550308,01,00
3,Loja 100,0000,122019,Master Formula Farmacia de Manipulacao LTDA,71605265000757,143351809116,3550308,01,00
4,Loja 100,0000,012020,Master Formula Farmacia de Manipulacao LTDA,71605265000757,143351809116,3550308,01,00
...,...,...,...,...,...,...,...,...,...
694,Loja 71,0000,082020,Master Formula Farmacia de Manipulacao LTDA,71605265000323,145800771110,3550308,01,00
695,Loja 71,0000,092020,Master Formula Farmacia de Manipulacao LTDA,71605265000323,145800771110,3550308,01,00
696,Loja 71,0000,102020,Master Formula Farmacia de Manipulacao LTDA,71605265000323,145800771110,3550308,01,00
697,Loja 71,0000,112020,Master Formula Farmacia de Manipulacao LTDA,71605265000323,145800771110,3550308,01,00


In [64]:
for date in planilha['Data'].unique():
    print(date)

102019
112019
122019
072020
092020
112020
122020
012020
022020
042020
102020
052020
062020
032020
082020


In [65]:
df_0150_final['COD_MUN'] = pd.to_numeric(df_0150_final['COD_MUN'])
df_0150_final

C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\4256047958.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_0150_final['COD_MUN'] = pd.to_numeric(df_0150_final['COD_MUN'])


,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,751895505,GILDERSON AUGUSTO FONSECA EIRELI ME,1058,29028142000149,NaN,118720948110,3550308,092020
1,0150,000000470,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265002024,NaN,119507430112,3550308,092020
2,0150,000355698,CONDE ATACADISTA LTDA,1058,27849963000110,NaN,125415766116,3508504,092020
3,0150,000000499,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265004906,NaN,128210260116,3550308,092020
4,0150,000001604,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265005554,NaN,128970130118,3550308,092020
...,...,...,...,...,...,...,...,...,...
356,0150,000007953,JULIA COSMETICOS EIRELI,1058,10328983000143,NaN,645538983114,3549904,122020
357,0150,000002004,DHV DISTRIBUIDORA HOSPITALAR DO VALE LTDA,1058,23276873000127,NaN,645739349118,3549904,122020
358,0150,000000458,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000838,NaN,796260079110,3518800,122020
359,0150,000000456,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000676,NaN,796436256113,3518800,122020


In [66]:
def fill_missing_ie(group):
    non_null_ie = group.dropna()['IE']
    if not non_null_ie.empty:
        group['IE'] = non_null_ie.iloc[0]
    elif group['IE'].isnull().all():
        group['IE'] = np.nan
    return group

# Group by 'CNPJ' and apply the custom function
df_0150_final_new = df_0150_final.groupby('CNPJ', group_keys=False).apply(fill_missing_ie)

df_0150_final_new

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,751895505,GILDERSON AUGUSTO FONSECA EIRELI ME,1058,29028142000149,NaN,118720948110,3550308,092020
1,0150,000000470,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265002024,NaN,119507430112,3550308,092020
2,0150,000355698,CONDE ATACADISTA LTDA,1058,27849963000110,NaN,125415766116,3508504,092020
3,0150,000000499,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265004906,NaN,128210260116,3550308,092020
4,0150,000001604,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265005554,NaN,128970130118,3550308,092020
...,...,...,...,...,...,...,...,...,...
356,0150,000007953,JULIA COSMETICOS EIRELI,1058,10328983000143,NaN,645538983114,3549904,122020
357,0150,000002004,DHV DISTRIBUIDORA HOSPITALAR DO VALE LTDA,1058,23276873000127,NaN,645739349118,3549904,122020
358,0150,000000458,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000838,NaN,796260079110,3518800,122020
359,0150,000000456,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000676,NaN,796436256113,3518800,122020


In [67]:
df_0150_final = df_0150_final[df_0150_final['IE'].notnull()]
df_0150_final

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,751895505,GILDERSON AUGUSTO FONSECA EIRELI ME,1058,29028142000149,NaN,118720948110,3550308,092020
1,0150,000000470,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265002024,NaN,119507430112,3550308,092020
2,0150,000355698,CONDE ATACADISTA LTDA,1058,27849963000110,NaN,125415766116,3508504,092020
3,0150,000000499,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265004906,NaN,128210260116,3550308,092020
4,0150,000001604,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265005554,NaN,128970130118,3550308,092020
...,...,...,...,...,...,...,...,...,...
356,0150,000007953,JULIA COSMETICOS EIRELI,1058,10328983000143,NaN,645538983114,3549904,122020
357,0150,000002004,DHV DISTRIBUIDORA HOSPITALAR DO VALE LTDA,1058,23276873000127,NaN,645739349118,3549904,122020
358,0150,000000458,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000838,NaN,796260079110,3518800,122020
359,0150,000000456,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000676,NaN,796436256113,3518800,122020


In [68]:
df_0150_final.loc[(df_0150_final['CNPJ'].notnull()) & (df_0150_final['IE'].isnull())]

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA


In [69]:
df_0150_final['CNPJ'] = df_0150_final['CNPJ'].apply(lambda x: str(x).zfill(14) if len(str(x)) < 14 else str(x))
df_0150_final

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA
0,0150,751895505,GILDERSON AUGUSTO FONSECA EIRELI ME,1058,29028142000149,NaN,118720948110,3550308,092020
1,0150,000000470,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265002024,NaN,119507430112,3550308,092020
2,0150,000355698,CONDE ATACADISTA LTDA,1058,27849963000110,NaN,125415766116,3508504,092020
3,0150,000000499,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265004906,NaN,128210260116,3550308,092020
4,0150,000001604,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265005554,NaN,128970130118,3550308,092020
...,...,...,...,...,...,...,...,...,...
356,0150,000007953,JULIA COSMETICOS EIRELI,1058,10328983000143,NaN,645538983114,3549904,122020
357,0150,000002004,DHV DISTRIBUIDORA HOSPITALAR DO VALE LTDA,1058,23276873000127,NaN,645739349118,3549904,122020
358,0150,000000458,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000838,NaN,796260079110,3518800,122020
359,0150,000000456,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000676,NaN,796436256113,3518800,122020


In [70]:
# Função para verificar o dígito verificador de um CNPJ
def verifica_digito_verificador(cnpj):
    cnpj = [int(d) for d in str(cnpj) if d.isdigit()]  # Converte o CNPJ em uma lista de números inteiros
    if len(cnpj) != 14:
        return False  # CNPJ deve ter 14 dígitos

    # Calcula o primeiro dígito verificador
    soma = 0
    peso = [5, 4, 3, 2, 9, 8, 7, 6, 5, 4, 3, 2]
    for i in range(12):
        soma += cnpj[i] * peso[i]
    resto = soma % 11
    if resto < 2:
        digito_verificador1 = 0
    else:
        digito_verificador1 = 11 - resto

    # Calcula o segundo dígito verificador
    soma = 0
    peso.insert(0, 6)
    for i in range(13):
        soma += cnpj[i] * peso[i]
    resto = soma % 11
    if resto < 2:
        digito_verificador2 = 0
    else:
        digito_verificador2 = 11 - resto

    # Verifica se os dígitos verificadores são iguais aos dígitos originais
    return cnpj[-2] == digito_verificador1 and cnpj[-1] == digito_verificador2

# Exemplo de DataFrame
df = df_0150_final.copy()

# Aplica a função verifica_digito_verificador à coluna 'CNPJ' do DataFrame
df['Dígito_Verificador_Válido'] = df['CNPJ'].apply(verifica_digito_verificador)

# Exibe o DataFrame com a nova coluna
df

,REG,COD_PART,NOME,COD_PAIS,CNPJ,CPF,IE,COD_MUN,DATA,Dígito_Verificador_Válido
0,0150,751895505,GILDERSON AUGUSTO FONSECA EIRELI ME,1058,29028142000149,NaN,118720948110,3550308,092020,True
1,0150,000000470,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265002024,NaN,119507430112,3550308,092020,True
2,0150,000355698,CONDE ATACADISTA LTDA,1058,27849963000110,NaN,125415766116,3508504,092020,True
3,0150,000000499,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265004906,NaN,128210260116,3550308,092020,True
4,0150,000001604,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265005554,NaN,128970130118,3550308,092020,True
...,...,...,...,...,...,...,...,...,...,...
356,0150,000007953,JULIA COSMETICOS EIRELI,1058,10328983000143,NaN,645538983114,3549904,122020,True
357,0150,000002004,DHV DISTRIBUIDORA HOSPITALAR DO VALE LTDA,1058,23276873000127,NaN,645739349118,3549904,122020,True
358,0150,000000458,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000838,NaN,796260079110,3518800,122020,True
359,0150,000000456,MASTER FORMULA FARMACIA DE MANIPULACAO LTDA,1058,71605265000676,NaN,796436256113,3518800,122020,True


In [71]:
df['Dígito_Verificador_Válido'].unique()

array([ True])

In [72]:
unique_cod_items = planilha[planilha['Data'] == '102019']['COD_ITEM'].unique()
arquivo_1050['COD_ITEM'] = arquivo_1050['COD_ITEM'].astype(int)
filtered = arquivo_1050[arquivo_1050['DATA'] == '102019'][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
filtered

C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\641789374.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == '102019'][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]


,Unnamed: 0,REF,COD_ITEM,QTD_INI,ICMS_INI,SALDO_FINAL_MES_QTD,SALDO_FINAL_MES_ICMS,DATA
0,0,201910,18,2,10.19250864652384,1,5.109999999999999,102019
15,15,201910,26,2,37.29868389097859,3,57.83,102019
45,45,201910,59,2,17.74959840421907,2,20.73,102019
60,60,201910,75,14,25.53822565882482,9,18.48,102019
75,75,201910,125,1,7.165861505110827,1,8.95,102019
...,...,...,...,...,...,...,...,...
100920,100920,201910,933218,5,5.912930150667711,3,3.63,102019
100965,100965,201910,933622,3,84.67719160401802,3,86.11,102019
100980,100980,201910,933713,2,24.34211693651845,2,27.22,102019
100995,100995,201910,933747,1,5.309880980917175,2,10.42,102019


In [73]:
len(filtered['COD_ITEM'].unique())

4446

In [82]:
# Geração dos TXTs da CAT

for date in planilha['Data'].unique():
#for date in ['112019']:
    # Geração da 0000
    df_0000_ = df_0000[df_0000['DATA'] == date][df_0000['LOJA'] == f'Loja {loja}']
    df_0000_['IE'] = df_0000_['IE'].str.replace('-', '')
    df_0000_ = df_0000_.drop('LOJA', axis=1)
    
    # Geração da 0150

    # df_0150 has originally 8 columns
    df_0150_ = df_0150_final[df_0150_final['DATA'] == date]
    df_0150_ = df_0150_[df_0150_['CNPJ'].notnull()]
    df_0150_['IE'] = df_0150_['IE'].str.replace('-', '')
    df_0150_ = df_0150_[['REG', 'COD_PART', 'NOME', 'COD_PAIS', 'CNPJ', 'CPF',
                'IE', 'COD_MUN']]
    
    # Geração da 0200
    df_0200 = planilha[planilha['Data'] == date][['COD_ITEM','DESCRICAO','CODIGO_BARRA','UNIDADE','N C M','ALIQUOTA','CEST']].drop_duplicates()
    df_0200['COD_REG'] = ['0200']*(df_0200.shape[0])
    df_0200 = df_0200[['COD_REG', 'COD_ITEM','DESCRICAO','CODIGO_BARRA','UNIDADE','N C M','ALIQUOTA','CEST']]
    df_0200['ALIQUOTA'] = df_0200['ALIQUOTA'].astype(float).astype(int)
    df_0200['CODIGO_BARRA'] = df_0200['CODIGO_BARRA'].astype(str)
    df_0200['CODIGO_BARRA'] = df_0200['CODIGO_BARRA'].replace('nan', np.nan)
    df_0200.drop_duplicates(subset='COD_ITEM', keep='first', inplace=True)
    
    # Geração da 1050
    unique_cod_items = planilha[planilha['Data'] == date]['COD_ITEM'].unique()
    arquivo_1050['COD_ITEM'] = arquivo_1050['COD_ITEM'].astype(int)
    filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
    filtered

    df_1050 = filtered[['COD_ITEM','QTD_INI', 'ICMS_INI', 'SALDO_FINAL_MES_QTD', 'SALDO_FINAL_MES_ICMS']]
    
    df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
    df_1050['QTD_INI'] = df_1050['QTD_INI'].astype(float).astype(int).astype(str) + ',000'
    df_1050['ICMS_INI'] = df_1050['ICMS_INI'].astype(float).map(format_number)
    df_1050['SALDO_FINAL_MES_QTD'] = df_1050['SALDO_FINAL_MES_QTD'].astype(float).astype(int).astype(str) + ',000'
    df_1050['SALDO_FINAL_MES_ICMS'] = df_1050['SALDO_FINAL_MES_ICMS'].astype(float).map(format_number)
    
    df_1050 = df_1050[['COD_REG', 'COD_ITEM', 'QTD_INI', 'ICMS_INI', 'SALDO_FINAL_MES_QTD', 'SALDO_FINAL_MES_ICMS']]
    df_1050.rename(columns={'COD_REG': 'REG'}, inplace=True)
    
    # Geração da 1100
    df_1100_ = df_1100[df_1100['Data'] == date]
    df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(float).map(format_number)
    df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(float).map(format_number)
    df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(str).replace('nan', np.nan)
    df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(str).replace('nan', np.nan)
    df_1100_ = df_1100_[['COD_REG','CHV_DOC','DATA','NUM_ITEM','IND_OPER','COD_ITEM','CFOP','QTD_FIN','ICMS_TOT_PCAT','VLR_CONFR_PCAT','COD_LEGAL_PCAT']]
    
    # Geração do .txt da 0000
    dataframe_to_txt_with_crlf(df_0000_, f'Loja_{loja}/CAT_Loja_{loja}/teste/0000_{date}_v1.txt')
    
    # Geração do .txt da 0150
    dataframe_to_txt_with_crlf(df_0150_, f'Loja_{loja}/CAT_Loja_{loja}/teste/0150_{date}_v1.txt')
    
    # Geração do .txt da 0200
    dataframe_to_txt_with_crlf(df_0200, f'Loja_{loja}/CAT_Loja_{loja}/teste/0200_{date}_v1.txt')
    
    # Geração do .txt da 1050
    dataframe_to_txt_with_crlf(df_1050, f'Loja_{loja}/CAT_Loja_{loja}/teste/1050_{date}_v1.txt')
    
    # Geração do .txt da 1100
    dataframe_to_txt_with_crlf(df_1100_, f'Loja_{loja}/CAT_Loja_{loja}/teste/1100_{date}_v1.txt')
    
    # Geração de um arquivo .txt único,
    # contendo todas as informações das documentações
    # da 0000, 0150, 0200, 1050 e 1100
    # dentro do formato correto

    read_files = glob.glob(f"Loja_{loja}/CAT_Loja_{loja}/teste/*{date}_v1.txt")
    
    with open(f"Loja_{loja}/CAT_Loja_{loja}/cat_loja_{loja}_{date}_v1.txt", "wb") as outfile:
        for f in read_files:
            with open(f, "rb") as infile:
                outfile.write(infile.read())

C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_0000_ = df_0000[df_0000['DATA'] == date][df_0000['LOJA'] == f'Loja {loja}']
C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:37: SettingWithCopyWarning: 
A value is trying to be set on 

C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(str).replace('nan', np.nan)
C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:50: SettingWithCopyWarning: 
A value is try

C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:49: SettingWithCopyWarning: 
A value is trying 

C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['ICMS_INI'] = df_1050['ICMS_INI'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['SALDO_FINAL_MES_QTD'] = df_1050['SALDO_FINAL_MES_QTD'].astype(float).astype(int).astype(str) + ',000'
C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:40: SettingWithCopyWarning: 
A value 

C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_0000_ = df_0000[df_0000['DATA'] == date][df_0000['LOJA'] == f'Loja {loja}']
C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:37: SettingWithCopyWarning: 
A value is trying to be set on 

C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(str).replace('nan', np.nan)
C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:50: SettingWithCopyWarning: 
A value is try

C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['ICMS_TOT_PCAT'] = df_1100_['ICMS_TOT_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1100_['VLR_CONFR_PCAT'] = df_1100_['VLR_CONFR_PCAT'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:49: SettingWithCopyWarning: 
A value is trying 

C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['ICMS_INI'] = df_1050['ICMS_INI'].astype(float).map(format_number)
C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['SALDO_FINAL_MES_QTD'] = df_1050['SALDO_FINAL_MES_QTD'].astype(float).astype(int).astype(str) + ',000'
C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:40: SettingWithCopyWarning: 
A value 

C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_0000_ = df_0000[df_0000['DATA'] == date][df_0000['LOJA'] == f'Loja {loja}']
C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = arquivo_1050[arquivo_1050['DATA'] == date][arquivo_1050['COD_ITEM'].isin(unique_cod_items)]
C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1050['COD_REG'] = ['1050']*(df_1050.shape[0])
C:\Users\vinic\AppData\Local\Temp\ipykernel_15968\280812627.py:37: SettingWithCopyWarning: 
A value is trying to be set on 